In [1]:
from nustar_pysolar import planning
import astropy.units as u

In [2]:
fname = planning.download_occultation_times(outdir='../data/')
print(fname)

../data/NUSTAR.2017_075.SHADOW_ANALYSIS.txt


In [8]:
tstart = '2017-03-21T13:00:00'
tend = '2017-03-21T20:00:00'
orbits = planning.sunlight_periods(fname, tstart, tend)
pa = planning.get_nustar_roll(tstart, 0)
print("NuSTAR Roll angle for Det0 in NE quadrant: {}".format(pa))

NuSTAR Roll angle for Det0 in NE quadrant: 334.6683600695206 deg


In [14]:
offset = [10., 10.]*u.arcsec

for ind, orbit in enumerate(orbits):
    midTime = (0.5*(orbit[1] - orbit[0]) + orbit[0])
    sky_pos = planning.get_sky_position(midTime, offset)
    print("Orbit: {}".format(ind))
    print("Orbit start: {} Orbit end: {}".format(orbit[0].isoformat(), orbit[1].isoformat()))
    print('Aim time: {} RA (deg): {} Dec (deg): {}'.format(midTime.isoformat(), sky_pos[0], sky_pos[1]))
    print("")

Orbit: 0
Orbit start: 2017-03-21T13:35:00 Orbit end: 2017-03-21T14:36:20
Aim time: 2017-03-21T14:05:40 RA (deg): 0.8264595394919657 deg Dec (deg): 0.3610003398707718 deg

Orbit: 1
Orbit start: 2017-03-21T15:11:40 Orbit end: 2017-03-21T16:13:00
Aim time: 2017-03-21T15:42:20 RA (deg): 0.8876196709481257 deg Dec (deg): 0.3875073748782843 deg

Orbit: 2
Orbit start: 2017-03-21T16:48:20 Orbit end: 2017-03-21T17:49:40
Aim time: 2017-03-21T17:19:00 RA (deg): 0.9487781865088669 deg Dec (deg): 0.41401307512990226 deg

Orbit: 3
Orbit start: 2017-03-21T18:25:00 Orbit end: 2017-03-21T19:26:20
Aim time: 2017-03-21T18:55:40 RA (deg): 1.009935111409283 deg Dec (deg): 0.4405174103958643 deg

